In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization

from sklearn.model_selection import train_test_split

In [2]:
info = pd.read_csv("_info.csv")
info

,FileID,Negative,Typical,Atypical
0,0003b2210c64,0,1,0
1,00af6f8c2a3d,1,0,0
2,00c9033fbc2e,0,0,1
3,00e0ce73dac8,0,0,1
4,00f0a591f18a,0,1,0
...,...,...,...,...
1195,ff60a32aa049,0,0,1
1196,ff9666e69d19,0,1,0
1197,ff971972490d,1,0,0
1198,ffb804652089,1,0,0


In [3]:
y_OneHot = np.array(info[['Negative','Typical','Atypical']])
y_OneHot

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1]])

In [4]:
fileids = info['FileID'].tolist()
X = []
for fileid in fileids:
    pixel_array = plt.imread("img/train/{}.jpg".format(fileid))
    pixel_array = cv2.resize(pixel_array, (256,256))   # some size
    X.append([pixel_array])
    
X = np.array(X)
X = X.astype("float16")
# 檢查
print(X.shape)

(1200, 1, 256, 256)


# 訓練集 測試集

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y_OneHot, test_size=0.2, random_state=42)

In [6]:
def show_loss(_history):
    loss = _history.history['loss']
    val_loss = _history.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc="upper right")
    plt.show()
    
    # accuracy
    score = model.evaluate(X_test, y_test)
    print("test acc", score[1])
    

# CNN model

In [7]:
model = Sequential()
model.add(Conv2D(filters=512, kernel_size=(64,64), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(16,16), padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=256, kernel_size=(32,32), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(16,16), padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(16,16), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(16,16), padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=(8,8), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(8,8), padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# 平坦層
model.add(Flatten())
# 隱藏層
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
# 輸出層
model.add(Dense(3, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
_history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=100,
                    validation_split=0.2,
                    epochs=100,
                    verbose=2)

Epoch 1/100


In [ ]:
show_loss(_history)

In [ ]:
model.save("model/cnn_20211116.md")